In [5]:
import pandas as pd
all_patients = pd.read_csv('/Users/jinyanwei/Desktop/BP_Model/Data/mimic-iv-2.2/hosp/patients.csv.gz')
# subject_id	gender	anchor_age	anchor_year	anchor_year_group	dod  299712 rows × 6 columns
diagnoses_icd = pd.read_csv('/Users/jinyanwei/Desktop/BP_Model/Data/mimic-iv-2.2/hosp/diagnoses_icd.csv.gz')
# subject_id	hadm_id	seq_num	icd_code	icd_version 4756326 rows × 5 columns
d_icd_diagnoses = pd.read_csv('/Users/jinyanwei/Desktop/BP_Model/Data/mimic-iv-2.2/hosp/d_icd_diagnoses.csv.gz')
# icd_code	icd_version	long_title 109775 rows × 3 columns
patients = pd.read_csv('/Users/jinyanwei/Desktop/BP_Model/Jinyw_code/cvd/extraction_path.csv')
# Patients have signals 

In [6]:
import numpy as np
icd9cvds = [str(i) for i in range(390, 460)] #['390',...,'459']
#icd-10: begin with I
dual_icd_code = pd.read_csv('/Users/jinyanwei/Desktop/BP_Model/Jinyw_code/cvd/ICD10_dual_code.csv')
cvds_icd_code = pd.read_csv('/Users/jinyanwei/Desktop/BP_Model/Jinyw_code/cvd/ICD10_cvds_code.csv')
dual_list = [i for i in dual_icd_code['dual_code']] # ['A188', 'A328', ..., 'N185']
title_cvds_dict = cvds_icd_code.set_index('simple_title')['cvds_code'].to_dict() #{'pericarditis': 'I32','rheumatoid arthritis': 'I39',...}
cvds_title9 = pd.read_csv('/Users/jinyanwei/Desktop/BP_Model/Jinyw_code/cvd/cvds_title9.csv')
cvds_title9['cvds_code'] = cvds_title9['cvds_code'].astype(str)
cvds_title10 = pd.read_csv('/Users/jinyanwei/Desktop/BP_Model/Jinyw_code/cvd/cvds_title10.csv')
cvds_title_both = pd.read_csv('/Users/jinyanwei/Desktop/BP_Model/Jinyw_code/cvd/cvds_title_both.csv')
cvds_title_both['cvds_code9'] = cvds_title_both['cvds_code9'].astype(str)
#display(cvds_title9)

In [ ]:
#sub_id = 10014354#patients['subject_id'][10000] #10000032 #10014354
part_subject = pd.DataFrame()
for subject in range(len(patients)):
    sub_id = patients['subject_id'][subject]
    subject_df = diagnoses_icd[diagnoses_icd['subject_id'] == sub_id]
    #subject_df = subject_df.drop_duplicates(subset=['icd_code'])
    subject_df = subject_df.drop_duplicates(subset=['icd_code', 'icd_version'])
    #print(f'id:{sub_id}, length:{len(subject_df)}')
    #display(subject_df)

    subject_icd9 = subject_df[subject_df['icd_version'] == 9]
    #display(subject_icd9)
    subject_filtered9 = subject_icd9[subject_icd9['icd_code'].str.startswith(tuple(icd9cvds))]
    #display(subject_filtered9)
    subject_cvds9 = subject_filtered9.merge(d_icd_diagnoses, left_on=['icd_code', 'icd_version'], right_on=['icd_code', 'icd_version'], how='inner')
    subject_cvds9['cvds_code'] = [subject_cvds9['icd_code'][i][:3] for i in range(len(subject_cvds9))]
    subject_cvds9 = subject_cvds9.merge(cvds_title9, left_on='cvds_code', right_on='cvds_code', how='inner')
    #display(subject_cvds9)

    subject_icd10 = subject_df[subject_df['icd_version'] == 10]
    #display(subject_icd10)
    subject_diagnoses10 = subject_icd10.merge(d_icd_diagnoses, left_on=['icd_code', 'icd_version'], right_on=['icd_code', 'icd_version'], how='inner')
    #display(subject_diagnoses10)
    subject_cvds10_list = []
    for dss in range(len(subject_diagnoses10)):
        cvds_row = []
        if 'I' in subject_diagnoses10['icd_code'][dss]:
            cvds_row = list(subject_diagnoses10.iloc[dss])
            cvds_code = subject_diagnoses10['icd_code'][dss][:3]
            cvds_row.append(cvds_code)
        else:
            for keys in list(title_cvds_dict.keys()):
                if keys in subject_diagnoses10['long_title'][dss]:
                    cvds_row = list(subject_diagnoses10.iloc[dss])
                    cvds_code = title_cvds_dict[keys]
                    cvds_row.append(cvds_code)
                    break
        if len(cvds_row) != 0:
            subject_cvds10_list.append(cvds_row)
    subject_cvds10 = pd.DataFrame(subject_cvds10_list, columns=['subject_id', 'hadm_id', 'seq_num', 'icd_code', 'icd_version', 'long_title', 'cvds_code'])
    subject_cvds10 = subject_cvds10.merge(cvds_title10, left_on='cvds_code', right_on='cvds_code', how='inner')
    #display(subject_cvds10)
    subject_cvds = pd.concat([subject_cvds9, subject_cvds10], axis=0)
    #display(subject_cvds)
    part_subject = pd.concat([part_subject,subject_cvds], axis=0)
## icd_code data view: pd.concat([])
'''
subject_amount = len(part_subject.drop_duplicates('subject_id'))
icd_amount = len(part_subject.drop_duplicates('long_title'))
cvds_amount = len(part_subject.drop_duplicates('cvds_title'))
print(f'Total {subject_amount} subjects, {cvds_amount} types CVDs, {icd_amount} diseases, {len(part_subject)} diagnoses')
display(part_subject)
'''
part_subject.to_csv(f'/Users/jinyanwei/Desktop/BP_Model/Jinyw_result/cvds_subject.csv', index=False)
## Subject ID data view

In [16]:
subject_amount = len(part_subject.drop_duplicates('subject_id'))
icd_amount = len(part_subject.drop_duplicates('long_title'))
cvds_amount = len(part_subject.drop_duplicates('cvds_title'))
print(f'Total {subject_amount} subjects, {cvds_amount} types CVDs, {icd_amount} diseases, {len(part_subject)} diagnoses')
display(part_subject)

Total 154 subjects, 68 types CVDs, 320 diseases, 1400 diagnoses


,subject_id,hadm_id,seq_num,icd_code,icd_version,long_title,cvds_code,cvds_title
0,10014354,20900955,7,I110,10,Hypertensive heart disease with heart failure,I11,Hypertensive heart disease
1,10014354,20900955,8,I509,10,"Heart failure, unspecified",I50,Heart failure
2,10014354,22508257,4,I5032,10,Chronic diastolic (congestive) heart failure,I50,Heart failure
3,10014354,26486158,2,I5023,10,Acute on chronic systolic (congestive) heart f...,I50,Heart failure
4,10014354,27487226,4,I5033,10,Acute on chronic diastolic (congestive) heart ...,I50,Heart failure
...,...,...,...,...,...,...,...,...
8,19994379,23099193,18,I2510,10,Atherosclerotic heart disease of native corona...,I25,Chronic ischaemic heart disease
9,19994379,23099193,16,I455,10,Other specified heart block,I45,Other conduction disorders
10,19994379,23099193,20,I341,10,Nonrheumatic mitral (valve) prolapse,I34,Nonrheumatic mitral valve disorders
11,19994379,23099193,21,I340,10,Nonrheumatic mitral (valve) insufficiency,I34,Nonrheumatic mitral valve disorders
